# Column Generation Practice

It is critical for me to be well-versed here, and I've taken this week to code and build column generation models. I took the time to understand these models and it won't be long before I can apply this to the UROP.

## Cutting Stock Problem from JuMP

Tutorial is sourced from https://jump.dev/JuMP.jl/stable/tutorials/algorithms/cutting_stock_column_generation/

In [1]:
using JuMP

import GLPK
import SparseArrays

Cutting stock is a pretty famous problem and the most common problem used to introduce column generation. It is referenced in MIT optimization classes at 6.720 (Linear Optimization) lecture 21 and 15.083 (Integer Optimization) lectures 17 and 18.

Suppose we had metal rods of a certain length $W$, and customers demanded $d_i$ rods each with length $w_i$. (The width of the rod is irrelevant). What is the LEAST number of metal rods of length $W$ we'd need to use to successfully cut all the rods for all the customers?

The actual tutorial uses width of paper rolls, but that confused me, and the image used in Linear Optimization lecture is more akin to metal rods, so that's what I'm using. I kept the variables using W's instead of L's to fith the JuMP tutorial.

## Data Display

This initial function allows us to display the data for what we are doing.

In [27]:
# code used in JuMP for a better structure: struct is like classes

struct Piece # represents one of the many demands by the customer:
    w::Float64 #length of a PIECE cut from a metal rod
    d::Int # number of pieces that must be cut of the specified width
end

struct Data # represents all the demands of the customers, encoding a parameter for length too
    pieces::Vector{Piece} #all the different demanded vectors
    W::Float64 # the length of each metal rod
end

function Base.show(io::IO, d::Data)
    
    #print to set up beautiful text
    println(io, "Data for the cutting stock problem:")
    println(io, " W = $(d.W)")
    println(io, "with pieces:")
    println(io, "   i  w_i   d_i")
    println(io, "   ------------")
    
    #for each data piece, print a row in the table
    for (i, p) in enumerate(d.pieces)
        println(io, lpad(i, 4), " ", lpad(p.w, 5), " ", lpad(p.d, 3))
    end
    return
end

function get_data()
    #this is our custom data
    data = [
        75 38
        75 44
        75 30
        75 41 # 4 rows
        75 36
        53.8 33
        53 36
        51 41 # 8 rows
        50.2 35
        32.2 37
        30.8 44 # 11 rows
        29.8 49
        20.1 37
        16.2 36 # 14 rows
        14.5 42
        11 33
        8.6 47 # 17 rows
        8.2 25
        6.6 49
        5.1 42 # 18 rows
    ]
    return Data([Piece(data[i, 1], data[i, 2]) for i in axes(data, 1)], 100)
end

data = get_data()

Data for the cutting stock problem:
 W = 100.0
with pieces:
   i  w_i   d_i
   ------------
   1  75.0  38
   2  75.0  44
   3  75.0  30
   4  75.0  41
   5  75.0  36
   6  53.8  33
   7  53.0  36
   8  51.0  41
   9  50.2  35
  10  32.2  37
  11  30.8  44
  12  29.8  49
  13  20.1  37
  14  16.2  36
  15  14.5  42
  16  11.0  33
  17   8.6  47
  18   8.2  25
  19   6.6  49
  20   5.1  42


## Formulation for Direct Scenario

Formulation goes like this.

Suppose that we have a large number of rolls which we can use, call it $J$ (this is some giant number, by default we can set it to $\sum_i d_i$; in this case we'll use 1000). So roll number is variable $j$ from $1$ to $J = 1000$.

There are two decision variables. One of them is $y_j$ representing whether we use roll $j$ or not (1 if yes, 0 if no). This is a binary variable. The other is $x_{ij}$ representing, for particular demand number $i$, how many rod segments from roll $j$ we cut for pieces of size $w_i$. $i$ ranges from $1$ to the number of different demands. $x_{ij}$ is integer.

The objective is to MINIMIZE the total number of rolls used, which would be $\sum_j y_j$.

Constraints:

- $x_{ij}$ non-negative integer
- $y_j$ binary
- We cut at least the demand total: $\sum_j x_{ij} \geq d_i \ \forall i$
- We have enough space to cut what we cut for each metal rod: $\forall j, \ \sum_i w_i x_{ij} \leq W y_j$ (if $y_j = 0$, then we cut nothing; if $y_j$ is 1, then we can cut at most up to $W$ for that rod)

In [28]:
I = length(data.pieces)

J = 1000 # number of rolls

model = Model(GLPK.Optimizer)
@variable(model, x[1:I, 1:J] >= 0, Int, base_name = "cuts")
@variable(model, y[1:J], Bin, base_name = "rolls")

@constraint(model, [j in 1:J], sum(data.pieces[i].w * x[i, j] for i in 1:I) <= data.W * y[j], base_name = "cut_each_rod")
@constraint(model, [i in 1:I], sum(x[i, j] for j in 1:J) >= data.pieces[i].d, base_name = "cut_to_demand")

@objective(model, Min, sum(y[j] for j in 1:J))

model
#don't let it print! takes too long to format

A JuMP Model
Minimization problem with:
Variables: 21000
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 20 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 1000 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 20000 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 20000 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 1000 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: x, y

## Reformulation in Patterns

The raw optimization will take too long, so we column generate... first, let's find some feasible solution.

We can kinda cheat at the introduction, because one feasible solution would be to have 1 cut for each of the demand...meaning that $x_{ij} = 1$ specifically for $j = 1...d_1$ and $i=1$, then $x_{ij} = 1$ specifically for $j = d_1+1, ..., d_2$ with $i=2$, and so on, which is wasteful but it's feasible.

We can reformulate based on the following: Suppose we took a look at a specific roll $j$. What are the different ways for which we can cut this roll? For instance, looking at $i = 18$, one feasible solution might be $x_{18, j} = 19$, meaning we cut 19 pieces of demand number 18 into roll $j$.

Therefore, we can call each roll $j$ a pattern $p$, in which we decide how many pieces of demand type $i$ we carve into that rod. Each of these patterns is denoted as a $I$-dimensional vector. For example, $[0, 0, ..., 19]$ indicates the case in which demand number $18$ gets 19 pieces, but no other demand number gets any pieces.

But <b>a pattern is not exactly a roll</b>. A pattern is a pattern, and we can have many rolls or no rolls or 1 roll which uses that pattern. For instance, maybe for 2 rolls we use the $[0, 0, ..., 19]$ pattern, it's just that now, by iterating over patterns instead of rolls, we get rid of the awkwardness of having to store $j$ for a roll (all rolls are the same, isn't it a bit weird to have to use enumerated rolls?) and also the awkwardness of having to use a $J = 1000$ to hope that 1000 is large enough to avoid any issues.

In other words, a pattern is a vector of cuts, which can be taken up (or used, adopted, etc.) by a non-negative integer number of rolls. Then we reformulate $y_j$ (whether a roll is used) into $x_p$, denoting HOW MANY TIMES a pattern is used, while we will need to use indices within $p$, let's call it $a_{ip}$, denotes how many pieces of type $i$ we cut in roll number $p$.

I was confused on this for quite some time. But a pattern is not exactly a roll, it... I explained it and I won't belabor the point.

It would mean we can express our solution based on different possibilities of cutting patterns $p = 1, ..., P$ where $a_{i, p}$ indicates the number of pieces for demand $i$ to be cut in pattern $p$. Then our reformulation looks as follows:

MINIMIZE $\sum_p x_p$, where $p$ represents literally all possible cutting patterns (this will be a terrifically large number, yes!), the summation goes from $1$ to $P$! (and there are a lot of cutting pattern vectors, for instance $[0, 0, ..., 0, 1, 1, 1, 1, 1, 1, 1]$ is another feasible one because 16+15+11+8.6+8.2+6.6+5.1 is 70.5 which is small enough to fit within one roll of length $W = 100$).

Constraints: Integer $x_p \geq 0$.

Constraints: $\sum_p a_{ip}x_p \geq d_i \ \forall i$. Must satisfy demand.

Let's write this as one program:

MINIMIZE $\sum_{p=1}^P x_p$

ST $\sum_{p=1}^P a_{ip}x_p \geq d_i \ \forall i = 1, ..., I$ (call this Eq. 1)

$x_p \geq 0, x_p \in \mathbb{Z}$

In particular, suppose we had some columns (i.e. possible $P$, a subset of the original $P$ which we will call $P_0$ which can be thought of as a vector of certain $p$'s we have chosen to be acceptable). How will we find a better column (i.e. new $p$ to be added)? 

Column generation: Relax integrality on $x$ and consider dual variable $\pi_i$ associated with demand constraint $i$. In other words, in Eq. 1, each constraint would be its standalone, and then we would have a dual constraint associated with it. Specifically, this is shadow prices. If we increase demand for length type $i$ by 1, then we will need $\pi_i$ rods.

One equation would be $\sum_{p=1}^P a_{1, p}x_p \geq d_1$.

## Considering the Dual

Then in the dual, remember that the dual swaps around the $c$ and $b$ matrices. Just a reminder: if we had

MIN $3x_1 + 4x_2$

such that $5x_1 + 6x_2 \geq 7$ (first primal constraint: dual variable $y_1$)

and $8x_1 + 9x_2 \geq 10$ (second primal constraint: dual variable $y_2$)

then our dual would be formed by

MAX $7y_1 + 10y_2$

such that $5y_1 + 8y_2 \leq 3$ (first dual constraint: use first COLUMN of matrix A and first coefficient of objective)

and $6y_1 + 9y_2 \leq 4$ (second dual constraint: use second COLUMN Of matrix A and second coefficient of objective)

That means if our PRIMAL equations were formed by:

$a_{1, 1}x_1  + a_{1, 2}x_2 + \dots + a_{1, P}x_P \geq d_1$

$a_{2, 1}x_1  + a_{2, 2}x_2 + \dots + a_{2, P}x_P \geq d_2$

$a_{3, 1}x_1  + a_{3, 2}x_2 + \dots + a_{3, P}x_P \geq d_3$...

($A$ would be a $I$ by $P$ matrix)

and our objective is MIN $x_1 + x_2 + ...$ (this is a $1$ by $P$ matrix; all coefficients are $1$)

then our DUAL would be constructed as follows:

Since we have $I$ constraints for the primal, we would have $I$ dual variables, call them $\pi_i$.

We would have $P$ constraints:

$a_{1, 1} \pi_1 + a_{2, 1} \pi_2 + \dots + a_{I, 1} \pi_I \leq 1$

$a_{1, 2} \pi_1 + a_{2, 2} \pi_2 + \dots + a_{I, 2} \pi_I \leq 1$

$a_{1, 3} \pi_1 + a_{2, 3} \pi_2 + \dots + a_{I, 3} \pi_I \leq 1$...

and our objective is MAX $\pi_1 d_1 + ... + \pi_I d_I$ (this is a $1$ by $I$ matrix)

To summarize:

Let $x$ be our decision variables, a column vector of $P$ by 1.

Then our matrix $A$ is a $I$ by $P$ matrix, multiply $Ax$ is $I$ by $P$ times $P$ times $1$ = 

$b$ which is a $I$ by $1$ column vector.

And our matrix $c$ is $1$ by $P$, also column (we use MIN $c^Tx$).

Meanwhile in the dual,

we let $\pi$ be our decision variables, a column vector of $I$ by 1.

Then our matrix $A^T$ is $I$ by $P$, and then our "$b$" in the dual will be $c$.

And for our objective MAX, we use $b^T$.

So to summarize, our old primal formulation is

MINIMIZE $\sum_{p=1}^P x_p$

ST $\sum_{p=1}^P a_{ip}x_p \geq d_i \ \forall i = 1, ..., I$ (call this Eq. 1)

$x_p \geq 0, x_p \in \mathbb{Z}$

and our new dual formulation is

MAX $\sum_{i=1}^I \pi_i d_i$

such that

$\forall p = 1, ..., P, \ \sum_{i=1}^I a_{i, p} \pi_i \leq 1$

One can compare this to 6.720 lecture 21 slide 12, in which $c_p$ is $1$ in this case.

## Prepare the Dual

Anyways, back to the economic interpretation. We note that $\pi_i$ is equivalent to how many rolls we can save, so we want this to be as big as possible. Now, we will change our dual formulation a bit. 

We want to add columns (i.e. new patterns) to our original set $P_0$ to make it bigger. So suppose we looked for a certain potential pattern, call it $y$. Then this has $I$ elements. We want to maximize the savings possible for a certain new potential pattern, so this means we maximize $\sum_{i=1}^I \pi_i y_i$. The only true constraint is that we need to make sure such a pattern is cuttable, so similar to how we forced feasibility with our original partitions (which we assumed to be valid), we will need that same constraint here:

$\sum_{i=1}^I w_i y_i \leq W$. Basically, for any vector of a partition/pattern, make sure that it is actually cuttable for that one rod.

And of course we need the constraint that $y$ is a non-negative integer vector.

This problem is the <b>pricing problem</b>. We want the MAX to have value greater than 1, otherwise, it was not worth adding the roll. The reason why is because all the coefficients in the dual on the RHS were 1.

## Solving the Subproblem: Pricing Problem

In [29]:
function solve_pricing(data::Data, π::Vector{Float64})
    I = length(π)
    model = Model(GLPK.Optimizer)
    @variable(model, y[1:I] >= 0, Int)
    @constraint(model, sum(data.pieces[i].w * y[i] for i in 1:I) <= data.W)
    @objective(model, Max, sum(π[i] * y[i] for i in 1:I))
    optimize!(model)
    
    # this will allow us to generate some solutions
    if objective_value(model) > 1
        #solution is worth adding
        println("Better solution found! It has a pattern vector of $(y)!")
        return round.(Int, value.(y)) # might have some decimal residues
    end
    return nothing
end

solve_pricing (generic function with 1 method)

## Choose some Trivial Example for Initializing $P_0$

In [30]:
# Just...patterns which cut enough for the demand... so be lazy, choose I total patterns
#this is feasible because have enough of the cut and we're through!
lazy_p0 = Vector{Int}[]

for i in 1:I
    pattern = zeros(Int, I)
    pattern[i] = floor(Int, min(data.W / data.pieces[i].w, data.pieces[i].d))
    push!(lazy_p0, pattern) 
end

P = length(lazy_p0)

20

In [31]:
println("Shape of Patterns:\n")
for row in lazy_p0
    println(row)
end

Shape of Patterns:

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0]
[0, 

In [32]:
# Alternatively, see non-zero elements

SparseArrays.sparse(hcat(lazy_p0...))

20×20 SparseArrays.SparseMatrixCSC{Int64, Int64} with 20 stored entries:
⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄

Note that our initial set $P_0$ is a feasible set of $I$ different patterns...the true $P$ is likely to be much larger obviously.

## Putting it All Together

### Initial Stage: Model

Follow the initial formulation PRIMAL:

MINIMIZE $\sum_{p=1}^P x_p$

ST $\sum_{p=1}^P a_{ip}x_p \geq d_i \ \forall i = 1, ..., I$ (call this Eq. 1)

$x_p \geq 0, x_p \in \mathbb{Z}$

In [46]:
model = Model(GLPK.Optimizer)
@variable(model, x[1:P] >= 0)
@objective(model, Min, sum(x))
@constraint(model, demand[i=1:I], lazy_p0[i]' * x == data.pieces[i].d)
model

A JuMP Model
Minimization problem with:
Variables: 20
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 20 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 20 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: demand, x

### Perform Column Generation

In [47]:
while true
    
    #first, get the PRIMAL solution
    println("\nSolving the PRIMAL with a lazy_p0 of length $(length(lazy_p0))")
    optimize!(model)
    
    #then, receive the dual variable
    println("Model SOLVED.")
    π = dual.(demand) #this is a 1 by I vector
    println("The dual vector is $(π).")
    
    # FIND A NEW SOLUTION
    new_pattern = solve_pricing(data, π)
    println("The new solution we found was $(new_pattern).")
    
    if new_pattern === nothing
        break
    end
    
    #otherwise we have something
    push!(lazy_p0, new_pattern)
    println("We found something! Now the new P_0 is going to have length $(length(lazy_p0))!")
    
    #=
    Initially, we had a variable x which had length P: there was x_1 through x_P.
    Now that we added one more column, we are going to include another variable for x.
    Note that this is the RELAXATION version so there is no integer constraint.
    =#
    push!(x, @variable(model, lower_bound = 0))
    
    #=
    In the objective so far, we have x_1 + ... + x_P. But after the new addition,
    we also need to add this new variable in x to the objective. To do so, 
    this is the method to use.
    =#
    set_objective_coefficient(model, x[end], 1.0)
    
    #=
    Finally, in the matrix of coefficients in the demand, for the final
    equation which looks like
    sum_i x_pnew * a_(i, pnew) >= demand_i
    
    we need the last x to have the coefficient for a_(i, pnew) to be
    new_pattern[i]
    =#
    for i in 1:I
        if new_pattern[i] > 0
            set_normalized_coefficient(demand[i], x[end], new_pattern[i])
        end
    end
end


Solving the PRIMAL with a lazy_p0 of length 86
Model SOLVED.
The dual vector is [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.25, 0.16666666666666666, 0.16666666666666666, 0.1111111111111111, 0.09090909090909091, 0.08333333333333333, 0.06666666666666667, 0.05263157894736842].
Better solution found! It has a pattern vector of VariableRef[y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9], y[10], y[11], y[12], y[13], y[14], y[15], y[16], y[17], y[18], y[19], y[20]]!
The new solution we found was [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0].
We found something! Now the new P_0 is going to have length 87!

Solving the PRIMAL with a lazy_p0 of length 87
Model SOLVED.
The dual vector is [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, -0.04545454545454547, 0.16666666666666666, 0.16666666666666666, 0.1111111111111111, 0.09090909090909091, 0.0833333333333333

Model SOLVED.
The dual vector is [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9290780141843973, 0.7659574468085107, 0.8368794326241138, 0.3333333333333333, 0.17021276595744683, 0.24113475177304955, 0.09219858156028349, -0.09929078014184411, -0.04964539007092206, -0.08510638297872364, 0.04964539007092196, 0.07801418439716334, -0.08510638297872342, -0.02836879432624112].
Better solution found! It has a pattern vector of VariableRef[y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9], y[10], y[11], y[12], y[13], y[14], y[15], y[16], y[17], y[18], y[19], y[20]]!
The new solution we found was [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0].
We found something! Now the new P_0 is going to have length 101!

Solving the PRIMAL with a lazy_p0 of length 101
Model SOLVED.
The dual vector is [1.0, 1.0, 1.0, 1.0, 1.0, 0.6133333333333334, 0.5688888888888888, 0.5244444444444444, 0.528888888888889, 0.3333333333333333, 0.28888888888888886, 0.33333333333333337, 0.19555555555555554, 0.14222222222222236

In [35]:
println(length(lazy_p0))

42


In [36]:
SparseArrays.sparse(hcat(lazy_p0...))

20×42 SparseArrays.SparseMatrixCSC{Int64, Int64} with 90 stored entries:
⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀
⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⡀⣀⡀⠠⠒⡀⠢⠊⢁⠁⠌
⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⢈⢀⡨⠡⠀⠚⢄⠂⠠⠀⠂
⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠡⠄⡀⠐⢌⢂⠀⠐⡀⠠⠌
⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⢁⣈⢊⠅⠱⠠⠤⠊⠈⢂⠀

This was a success. We were able to generate 22 new solutions.

In [37]:
# example pattern: 25
for i in 1:I
    if lazy_p0[25][i] > 0
        println(lazy_p0[25][i], " units of piece $i")
    end
end

1 units of piece 8
1 units of piece 12
1 units of piece 16
1 units of piece 18


Also view what $x$ is.

In [48]:
println(value.(x))

[34.5, 44.0, 30.0, 41.0, 7.041666666666668, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 35.0, 0.0, 0.0, 0.7499999999999994, 0.0, 0.0, 19.25, 0.0, 0.0, 11.75, 0.0, 11.25, 13.25, 21.75, 5.0, 16.125, 13.5, 0.8333333333333334, 3.5, 3.5, 7.0]


## Final Step: Re-examine Solution

Note that we had relaxed the integer constraint on $x$, i.e. it was a linear relaxation. To force integer feasibility, round up on all the patterns as a simple heuristic.

In [49]:
for p in 1:length(x)
    num_rolls = ceil(Int, value(x[p]))
    if num_rolls > 0
        println(lpad(num_rolls, 2), " rolls of pattern $p")
    end
end

35 rolls of pattern 1
44 rolls of pattern 2
30 rolls of pattern 3
41 rolls of pattern 4
 8 rolls of pattern 5
15 rolls of pattern 21
35 rolls of pattern 22
 1 rolls of pattern 25
20 rolls of pattern 28
12 rolls of pattern 31
12 rolls of pattern 33
14 rolls of pattern 34
22 rolls of pattern 35
 5 rolls of pattern 36
17 rolls of pattern 37
14 rolls of pattern 38
 1 rolls of pattern 39
 4 rolls of pattern 40
 4 rolls of pattern 41
 7 rolls of pattern 42


In [50]:
sum(ceil.(Int, value.(x)))

341

## Answer is 341...unless we force integrality first and then optimize

In [51]:
set_integer.(x)
optimize!(model)
total_rolls = sum(ceil.(Int, value.(x)))
println("Total rolls needed if we force integrality first: $total_rolls")

Total rolls needed if we force integrality first: 334


Warning: Not necessarily global minimum, because in solving the MIP, we don't add columns... we first used a relaxation, then column generation, then forced integrality to see what would happen when we'd already added the columns.